In [1]:
import os

os.environ['AWS_ENDPOINT_URL'] = 'https://s3.us-west-000.backblazeb2.com'
os.environ['AWS_ACCESS_KEY_ID'] = ''
os.environ['AWS_SECRET_ACCESS_KEY'] = ''

In [2]:
from tqdm import tqdm
import boto3
import subprocess
from unidecode import unidecode
import IPython.display as ipd
from glob import glob

os.makedirs('imda-tts', exist_ok = True)
os.makedirs('imda-tts-audio', exist_ok = True)

In [3]:
session = boto3.Session()

In [4]:
s3 = session.resource('s3')
bucket = s3.Bucket('imda-singapore')

In [5]:
for f in tqdm(bucket.objects.filter(Prefix='PART1/TTS/CHANNEL0/SCRIPT')):
    filename = os.path.join('imda-tts', os.path.split(f.key)[1])
    if os.path.exists(filename):
        continue
                            
    bucket.download_file(f.key, filename)

1it [00:01,  1.42s/it]


In [23]:
for f in tqdm(bucket.objects.filter(Prefix='PART1/TTS/CHANNEL0/WAVE')):
    filename = os.path.join('imda-tts-audio', os.path.split(f.key)[1])
    if os.path.exists(filename):
        continue
                            
    bucket.download_file(f.key, filename)

1it [00:24, 24.79s/it]


In [24]:
!ls imda-tts-audio

FEMALE_01  FEMALE_01.zip


In [25]:
!rm -rf imda-tts-audio/FEMALE_01

In [26]:
!ls imda-tts

FEMALE_01.txt


In [27]:
with open('imda-tts/FEMALE_01.txt') as fopen:
    text = fopen.read().split('\n')
len(text)

6033

In [28]:
text[0]

'0000\tAuthor of the danger trail, Philip Steels, etc.'

In [33]:
filtered = []
for t in text:
    splitted = t.split('\t')
    if len(splitted) == 2:
        filename = f'{splitted[0]}.wav'
        t = splitted[1]
        filtered.append({
            'filename': filename,
            'text': t,
        })
        
filtered[1]

{'filename': '0001.wav',
 'text': 'Not at this particular case, Tom, apologized Whittemore.'}

In [35]:
import json

with open('texts.json', 'w') as fopen:
    json.dump(filtered, fopen)

In [29]:
from huggingface_hub import HfApi
api = HfApi()

In [30]:
api.upload_file(
    path_or_fileobj='imda-tts-audio/FEMALE_01.zip',
    path_in_repo='FEMALE_01.zip',
    repo_id='mesolitica/IMDA-TTS',
    repo_type='dataset',
)

FEMALE_01.zip:   0%|          | 0.00/1.82G [00:00<?, ?B/s]

'https://huggingface.co/datasets/mesolitica/IMDA-TTS/blob/main/FEMALE_01.zip'

In [36]:
api.upload_file(
    path_or_fileobj='texts.json',
    path_in_repo='texts.json',
    repo_id='mesolitica/IMDA-TTS',
    repo_type='dataset',
)

'https://huggingface.co/datasets/mesolitica/IMDA-TTS/blob/main/texts.json'